* Weather resource: https://www.timeanddate.com/weather/@2176947
* Webscraping resource: https://stackoverflow.com/questions/51756775/scraping-table-from-website-timeanddate-com
* Install Chromedriver: https://chromedriver.chromium.org/downloads

In [1]:
# !pip install selenium=4.2.0

In [2]:
import os
import pandas as pd
import requests, re, typing
from bs4 import BeautifulSoup as soup
import contextlib
from selenium import webdriver #need to install chromedriver to iterate through day filter
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

c:\Users\Carl\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
base_dir = os.path.dirname(os.path.abspath("..\code"))
path = os.path.join(base_dir, r"code")
os.chdir(path)

In [4]:
def _remove(d:list) -> list:
    return list(filter(None, [re.sub('\xa0', '', b) for b in d]))

In [5]:
@contextlib.contextmanager
def get_weather_data(url:str, by_url = True) -> typing.Generator[dict, None, None]:
   d = soup(requests.get(url).text if by_url else url, 'html.parser')
   _table = d.find('table', {'id':'wt-his'})
   _data = [[[i.text for i in c.find_all('th')], *[i.text for i in c.find_all('td')]] for c in _table.find_all('tr')]
   [h1], [h2], *data, _ = _data
   _h2 = _remove(h2)
   yield {tuple(_remove(h1)):[dict(zip(_h2, _remove([a, *i]))) for [[a], *i] in data]}

In [6]:
def get_weather_dict(chrome_dir, url):
    d = webdriver.Chrome(chrome_dir)
    d.get(url)
    _d = {}
    for i in d.find_element_by_id('wt-his-select').find_elements_by_tag_name('option'):
        i.click()
        with get_weather_data(d.page_source, False) as weather:
            _d[i.text] = weather
    d.close()
    return _d

In [7]:
def get_daily_humidty(_d):
    df_all = pd.DataFrame()
    for date, dict in _d.items():
        for day in dict.values():
            df = pd.DataFrame(day)
            df.columns = ["HUMIDITY","WINDDIRECTION", "TEMP", "TIME", "BAROMETER", "WEATHER", "WINDSPEED"]
            df["DATE"] = date
            df["TIME"] = df["TIME"].str.split('m', expand=True)[0] + "m"
            df_all = df_all.append(df)
    return df_all

In [8]:
this_year = int(datetime.today().strftime('%Y'))
this_month = int(datetime.today().strftime('%m')) 

In [9]:
base_dir = os.path.dirname(os.path.abspath("..\data"))
dest = os.path.join(base_dir, r"data") + r"\\Weather - timeanddate"
print(dest)

c:\Users\Carl\Desktop\UNSW Capstone Project\GitHub Capstone Project\GroupF_Project\data\\Weather - timeanddate


In [11]:
start_year = 2017
end_year = 2022
df_summary = pd.DataFrame()
for year in range(start_year, end_year+1):
    if year > this_year: break
    for month in range(1, 13):
        if (year == this_year) & (month > this_month): break
        print(year, month)
        url = f'https://www.timeanddate.com/weather/@2176947/historic?month={month}&year={year}'
        chrome_dir = path + r'\Archive\chromedriver' #insert path to installed chromedriver (exe file)
        _d = get_weather_dict(chrome_dir, url)
        df = get_daily_humidty(_d)
        df_summary = df_summary.append(df) 
    file_name = f"weather - {year}.csv"
    df_summary.to_csv(dest + "\\" + file_name) 

2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
2021 1
2021 2
2021 3
2021 4
2021 5
2021 6
2021 7
2021 8
2021 9
2021 10
2021 11
2021 12
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
